## Dropout

`Dropout`就是随机丢掉网络输入种的特征，丢掉太多或者太少都不好，所以`dropout rate`一般设置为`0.3～0.5`。在`PyTorch`中有两种方法实现`dropout`，一种是以类的方式实现的：

```python
class Dropout(_DropoutNd):
```

其官网文档[torch.nn.Dropout(p=0.5, inplace=False)](https://pytorch.org/docs/master/generated/torch.nn.Dropout.html#torch.nn.Dropout)

另一种是以函数的方式实现的：

```python
def dropout(input: Tensor, p: float = 0.5, training: bool = True, inplace: bool = False) 
```

其官网文档[torch.nn.functional.dropout(input, p=0.5, training=True, inplace=False)](https://pytorch.org/docs/master/generated/torch.nn.functional.dropout.html#torch.nn.functional.dropout)

### 原理


- 原始论文：http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf

考虑一个带有$L$层隐藏层的神经网络，$l \in \{1, \cdots, L\}$。$z^{(l)}$定义为$l$层网络的输入向量，$y^{(l)}$定义为$l$层网络的输出向量，$w^{(l)}$定义为$l$层网络的权重，$b^{(l)}$定义为$l$层网络的偏置。标准的前馈神经网络可以描述为：

其中$f$为激活函数，比如$f(x)=1/(1+exp(-x))$。基于dropout，前馈网络可以表示为：

$$
\begin{aligned}
z_{i}^{(l+1)} &=\mathbf{w}_{i}^{(l+1)} \mathbf{y}^{l}+b_{i}^{(l+1)} \\
y_{i}^{(l+1)} &=f\left(z_{i}^{(l+1)}\right)
\end{aligned}
$$

添加完dropout之后的话，前馈网络可以表示为：

$$
\begin{aligned}
r_{j}^{(l)} & \sim \operatorname{Bernoulli}(p) \\
\widetilde{\mathbf{y}}^{(l)} &=\mathbf{r}^{(l)} * \mathbf{y}^{(l)}, \\
z_{i}^{(l+1)} &=\mathbf{w}_{i}^{(l+1)} \widetilde{\mathbf{y}}^{l}+b_{i}^{(l+1)} \\
y_{i}^{(l+1)} &=f\left(z_{i}^{(l+1)}\right)
\end{aligned}
$$

但是dropout实现由两种方式，一种是上面的`vanilla`版本，但是在做`inference`的时候，数据会进行$1-p$倍的缩小。`inverted dropout`在丢弃完之后，进行$1/(1-p)$倍的放大。反向传播的时候也是同步放大，但是在做`inference`的时候就不需要额外的处理了。

## 类方法实现的Dropout

类方法实现的源码中，其`Dropout`类是继承自`_DropoutNd`类, 而`_DropoutNd`类是继承`Module`类。

由于此种方式实现的`Dropout`是一个类，所以用的时候，我们首先需要实例化这个对象，然后将这个对象作用到一个输入上。实例化这个`Dropout`对象的时候，我们需要传入一个参数$p$，表示的是以概率$p$，从`Bernoulli`分布中采样，来作用到给定的输入元素中，也就是随机将输入元素以概率$p$屏蔽掉，也就是输入中不被`mask`为`0`的概率为$1-p$。

官方文档中还给了一篇论文：[Improving neural networks by preventing co-adaptation of feature detectors](https://arxiv.org/pdf/1207.0580.pdf)

因为最终只有$1-p$的概率的输入被保留下来了，所以在训练时，输出会被缩放到$\frac{1}{1-p}$倍，这种处理方式在做测试(推理)时就不需要额外的处理了。

`Python`示例如下：

```python
m = nn.Dropout(p=0.2)
input = torch.randn(20, 16)
output = m(input)
```

In [1]:
import torch
import numpy as np
import torch.nn as nn

随机生成一个四行五列的数据

In [2]:
data = torch.randn(4, 5)
data

tensor([[ 1.1388,  0.7845,  0.0998,  0.5277,  1.3779],
        [ 0.6497, -0.5809, -0.4241,  2.1075, -0.1612],
        [-0.2707,  0.2061, -1.6815, -0.4468, -0.2063],
        [-2.2523, -0.7526,  1.7570, -0.3444, -0.9290]])

将模型设置为训练模式：

In [3]:
m = nn.Dropout(p=0.5)  # 参数`p`表示不保留节点的比例。
m.train()

Dropout(p=0.5, inplace=False)

In [4]:
output = m(data)
output

tensor([[ 2.2776,  1.5690,  0.0000,  0.0000,  2.7557],
        [ 1.2994, -1.1618, -0.8482,  4.2151, -0.0000],
        [-0.0000,  0.4122, -0.0000, -0.0000, -0.0000],
        [-4.5046, -1.5052,  0.0000, -0.6887, -0.0000]])

将模型设置为测试模式：

In [5]:
m.eval()
output = m(data)
output

tensor([[ 1.1388,  0.7845,  0.0998,  0.5277,  1.3779],
        [ 0.6497, -0.5809, -0.4241,  2.1075, -0.1612],
        [-0.2707,  0.2061, -1.6815, -0.4468, -0.2063],
        [-2.2523, -0.7526,  1.7570, -0.3444, -0.9290]])

从上述输出结果可以看出，训练和测试阶段的不同之处了。训练时会对数据进行一个缩放，测试时是全部输出。

## 函数方法实现

在函数方法实现中，需要传入参数`input`，置`0`的概率`p`，`training`表示是否为测试。但是在类方法实现中，并不需要传入`training`这个参数，是因为它继承自`module`类，设置模型为`eval`时，就会将其内部属性`self.training`设置为`False`。

官方网址：[torch.nn.functional.dropout(input, p=0.5, training=True, inplace=False)](https://pytorch.org/docs/master/generated/torch.nn.functional.dropout.html#torch.nn.functional.dropout)

## Numpy实现

In [6]:
def dropout(x, p):
    """
    # dropout函数的实现，函数中，x是本层网络的激活值。p就是dropout就是每个神经元要被丢弃的概率。
    """
    if p < 0. or p >= 1: #p是概率值，必须在0~1之间
        raise ValueError('Dropout level must be in interval [0, 1[.')
    retain_prob = 1. - p
 
    # 我们通过binomial函数，生成与x一样的维数向量。binomial函数就像抛硬币一样，我们可以把每个神经元当做抛硬币一样
    # 硬币 正面的概率为p，n表示每个神经元试验的次数
    # 因为我们每个神经元只需要抛一次就可以了所以n=1，size参数是我们有多少个硬币。
    #即将生成一个0、1分布的向量，0表示这个神经元被屏蔽，不工作了，也就是dropout了
    random_tensor = np.random.binomial(n=1, p=retain_prob, size=x.shape) 
    print("random_tensor: ", random_tensor)
 
    x *= random_tensor
    print("x: ", x)
    x /= retain_prob
 
    return x

In [7]:
#对dropout的测试，大家可以跑一下上面的函数，了解一个输入x向量，经过dropout的结果  
x=np.asarray([1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
dropout(x,0.4)

random_tensor:  [1 1 0 1 1 1 1 0 1 0]
x:  [1. 2. 0. 4. 5. 6. 7. 0. 9. 0.]


array([ 1.6666666,  3.3333333,  0.       ,  6.6666665,  8.333333 ,
       10.       , 11.666666 ,  0.       , 14.999999 ,  0.       ],
      dtype=float32)

而如果在整个网络计算过程中，其大体流程如下所示：

In [8]:
import numpy as np

In [9]:
def train(rate, x, w1, b1, w2, b2):
    layer1 = np.maximum(0, np.dot(w1, x)+b1)
    mask1 = np.random.binomial(1, 1-rate, layer1.shape)
    layer1 = layer1 * mask1
    
    layer2 = np.maximum(0, np.dot(w2, layer2)+b2)
    mask2 = np.random.binomial(1, 1-rate, layer2.shape)
    layer2 = layer2 * mask2
    
    return layer2

In [10]:
def test(rate, x, w1, b1, w2, b2):
    layer1 = np.maximum(0, np.dot(w1, x) + b1)
    layer1 = layer1 * (1 - rate)
    
    layer2 = np.maximum(0, np.dot(w2, layer1) + b2)
    layer2 = layer2 * (1 - rate)
    
    return layer2

上述计算方法是在训练和测试阶段两部分都做了缩放，为了方便模型之后部署，减少推理的计算时间，还有一种方法是全部在训练过程中做缩放，而测试部分不动：

In [11]:
def another_train(rate, x, w1, b1, w2, b2):
    layer1 = np.maximum(0, np.dot(w1, x)+b1)
    mask1 = np.random.binomial(1, 1-rate, layer1.shape)
    layer1 = layer1 * mask1
    layer1 = layer1 / (1 - rate)
    
    layer2 = np.maximum(0, np.dot(w2, layer2)+b2)
    mask2 = np.random.binomial(1, 1-rate, layer2.shape)
    layer2 = layer2 * mask2
    layer2 = layer2 / (1 - rate)
    
    return layer2

In [12]:
def another_test(rate, x, w1, b1, w2, b2):
    layer1 = np.maximum(0, np.dot(w1, x) + b1)
    
    layer2 = np.maximum(0, np.dot(w2, layer1) + b2)
    
    return layer2